In [12]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df_sch, meta_sch = pyreadstat.read_sav('data/CY08MSP_STU_QQQ_HUN.sav')

In [14]:
import os
with open('data/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [17]:
   
cols = utils.meta2docs(meta_sch, excluded=['test','CNTRYID'])

cols_vectorstore = FAISS.from_documents(cols, OpenAIEmbeddings())

cols_retriever = cols_vectorstore.as_retriever()

In [10]:
meta_sch

In [7]:
df_sch2, meta_sch2 = pyreadstat.read_sav('data/PISA2022_SCH_QQQ.SAV')
cols2 = utils.meta2docs(meta_sch2, excluded=['test', 'CNTRYID'])

PyreadstatError: File data/PISA2022_SCH_QQQ.SAV does not exist!

In [ ]:
len(meta_sch.column_names)

In [ ]:
len(cols)


In [14]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give linear regression coefficients as answer.', df_sch, meta_sch, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['MATHMOT', 'MATHPREF', 'ST296Q01JA', 'ST296Q02JA']
content="```python\nimport pandas as pd\nimport numpy as np\nfrom utils import pv_reg_math\n\ndef run(data: pd.DataFrame) -> float:\n    data = data[['ESCS', 'SOCONPA', 'MATHMOT', 'MATHPREF']].dropna()\n    independent_vars = ['ESCS', 'SOCONPA']\n    regression_coefficients = pv_reg_math(independent_vars)\n    \n    return float(regression_coefficients)\n```" response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 356, 'total_tokens': 449}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1357fd1a-e082-4167-a758-ed84ca561853-0' usage_metadata={'input_tokens': 356, 'output_tokens': 93, 'total_tokens': 449}

import pandas as pd
import numpy as np
from utils import pv_reg_math

def run(data: pd.DataFrame) -> float:
    data = data[['ESCS', 'SOCONPA'

ImportError: cannot import name 'pv_reg_math' from 'utils' (/home/jovyan/work/Arpad/Tabular/utils.py)

In [23]:
test_data = [
    {'question':'Which country has the highest average  teacher student ratio? How much is it? ','expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 28.95},
    {'question':'Which country has the highest average class sizes? How much is it?','expected_columns': ['CLSIZE', 'CNT'], 'expected_answer': 42.65 },    
    {'question':'In how many languages ​​was the questionnaire filled out?', 'expected_columns': ['LANGTEST_QQQ'], 'expected_answer': 54},
    {'question':'What is Hungary average teacher-student ratio?', 'expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 10.83},
    {'question':'What is OECD average teacher-student ratio?', 'expected_columns': ['STRATIO', 'OECD'], 'expected_answer':  12.551608},
    {'question':'Where is average teacher-student ratio in Hungary compared to the OECD average in terms of average teacher-student ratio? Give the difference as number.','expected_columns': ['STRATIO', 'OECD', 'CNT'], 'expected_answer': -1.721608 },    
    #{'question':'how many education systems were involved in the survey? ','expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 28.95},
    {'question':'how many countries were involved in the survey?','expected_columns': ['CNT'], 'expected_answer': 80},
    {'question':'how many countries that are OECD memeber were involved in the survey? ','expected_columns': ['OECD', 'CNT'], 'expected_answer': 37},
    {'question':'how many schools were involved in the survey?','expected_columns': ['CNTSCHID'], 'expected_answer': 21629},
    {'question':'what is the relation between school autonomy  and teacher participation? Please give linear regression coefficient as result','expected_columns': ['SCHAUTO','TCHPART'], 'expected_answer':  0.2216 },
]


In [ ]:
test_data  = [
    {'question':'How many students were interviewed?','expected_columns': [ 'CNTSTUID','ST255Q01JA'], 'expected_answer': 613744},
    {'question':'How many countries where in the survey?','expected_columns': [ 'CNT'], 'expected_answer': 88},
    {'question':'In which country have students the most books at home? How much is it?','expected_columns': [ 'CNT','ST255Q01JA'], 'expected_answer': 0},    
    {'question':'What is the relation between social background and math score of students? Please give linear regression coefficients as answer.','expected_columns': ['ESCS', 'PV1MATH', 'PV2MATH', 'PV3MATH', 'PV4MATH', 'PV5MATH', 'PV6MATH', 'PV7MATH', 'PV8MATH', 'PV9MATH', 'PV10MATH'],  'expected_answer': 0],
]

In [68]:
utils.pipeline(llm, 'In which country have students the most books at home? How much is it?', df_sch, meta_sch, cols_retriever)

Country, Number of books at home
hint Country
['REGION', 'OECD', 'COBN_F', 'CNT']
hint  Number of books at home
['PA195Q01JA', 'ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA']
content="def run(data):\n    data.dropna(subset=['CNT', 'ST255Q01JA'], inplace=True)\n    max_books = data.groupby('CNT')['ST255Q01JA'].sum().idxmax()\n    total_books = data.groupby('CNT')['ST255Q01JA'].sum().max()\n    \n    return float(total_books)" response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 318, 'total_tokens': 387}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3399eb83-40fe-439a-8cee-d13c5ed21378-0' usage_metadata={'input_tokens': 318, 'output_tokens': 69, 'total_tokens': 387}
def run(data):
    data.dropna(subset=['CNT', 'ST255Q01JA'], inplace=True)
    max_books = data.groupby('CNT')['ST255Q01JA'].sum().idxmax()
    total_books = data.groupby('CNT')['ST255Q01JA'].sum().max()
    
    return float(total_books)


{'question': 'In which country have students the most books at home? How much is it?',
 'result': 36617.0,
 'hint_cols': ['REGION',
  'OECD',
  'COBN_F',
  'CNT',
  'PA195Q01JA',
  'ST255Q01JA',
  'ST256Q10JA',
  'ST256Q06JA']}

In [72]:
utils.pipeline(llm, 'Which country have students the most books at home? How much is it in average?', df_sch, meta_sch, cols_retriever)

Country, Number of books at home
hint Country
['REGION', 'OECD', 'COBN_F', 'CNT']
hint  Number of books at home
['PA195Q01JA', 'ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA']
content="def run(data):\n    data = data.dropna(subset=['CNT', 'ST255Q01JA'])\n    avg_books_per_country = data.groupby('CNT')['ST255Q01JA'].mean()\n    max_avg_books = avg_books_per_country.max()\n    \n    return float(max_avg_books)" response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 319, 'total_tokens': 379}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6577ad3f-ff51-4154-963b-e80d34773fb2-0' usage_metadata={'input_tokens': 319, 'output_tokens': 60, 'total_tokens': 379}
def run(data):
    data = data.dropna(subset=['CNT', 'ST255Q01JA'])
    avg_books_per_country = data.groupby('CNT')['ST255Q01JA'].mean()
    max_avg_books = avg_books_per_country.max()
    
    return float(max_avg_books)


{'question': 'Which country have students the most books at home? How much is it in average?',
 'result': 4.803861625100563,
 'hint_cols': ['REGION',
  'OECD',
  'COBN_F',
  'CNT',
  'PA195Q01JA',
  'ST255Q01JA',
  'ST256Q10JA',
  'ST256Q06JA']}

In [ ]:

utils.pipeline_l2(llm, 'Which country have students the most books at home? How much is it in average?', )

In [75]:
utils.pipeline_l2(llm, 'Which country have students the most books at home? How much is it in average?', df_sch, meta_sch, cols_retriever)

'STUDENT'

In [76]:
utils.pipeline_l2(llm, 'Which country has the highest average  teacher student ratio? How much is it?', df_sch, meta_sch, cols_retriever)

'SCHOOL'

In [77]:
utils.pipeline_l2(llm, 'how many countries were involved in the survey?', df_sch, meta_sch, cols_retriever)

'NONE'

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [36]:
cols_retriever.invoke('1. Student ID')

[Document(page_content='Intl. Student ID', metadata={'year': 2022, 'original_col_name': 'CNTSTUID'}),
 Document(page_content='Intl. School ID', metadata={'year': 2022, 'original_col_name': 'CNTSCHID'}),
 Document(page_content="Students' age", metadata={'year': 2022, 'original_col_name': 'AGE'}),
 Document(page_content="Student's county of birth", metadata={'year': 2022, 'original_col_name': 'COBN_S'})]

In [37]:
utils.pipeline(llm, 'How many students were interviewed?', df_sch, meta_sch, cols_retriever)

1. Student ID
2. Interview Date
3. Interviewer Name
4. Interview Location
5. Interview Duration
6. Interview Notes
hint 1. Student ID
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
hint 2. Interview Date
['VER_DAT', 'ST330Q03WA', 'ST330Q02WA', 'Option_TQ']
hint 3. Interviewer Name
['Option_TQ', 'LANGTEST_QQQ', 'Option_PQ', 'Option_WBQ']
hint 4. Interview Location
['ST330Q02WA', 'Option_TQ', 'ST330Q03WA', 'REGION']
hint 5. Interview Duration
['DURECEC', 'Option_TQ', 'ST273Q05JA', 'LANGTEST_QQQ']
hint 6. Interview Notes
['LANGTEST_QQQ', 'Option_TQ', 'ST330Q02WA', 'PV6READ']
content="def run(df):\n    return float(df['CNTSTUID'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 782, 'total_tokens': 801}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b7dc621e-bcce-4465-8f65-015e9dce1c78-0' usage_metadata={'input_tokens': 782, 'output_tokens': 19, 'total_tokens': 801}
def run(df):
    

{'question': 'How many students were interviewed?',
 'result': 613744.0,
 'hint_cols': ['CNTSTUID',
  'CNTSCHID',
  'AGE',
  'COBN_S',
  'VER_DAT',
  'ST330Q03WA',
  'ST330Q02WA',
  'Option_TQ',
  'Option_TQ',
  'LANGTEST_QQQ',
  'Option_PQ',
  'Option_WBQ',
  'ST330Q02WA',
  'Option_TQ',
  'ST330Q03WA',
  'REGION',
  'DURECEC',
  'Option_TQ',
  'ST273Q05JA',
  'LANGTEST_QQQ',
  'LANGTEST_QQQ',
  'Option_TQ',
  'ST330Q02WA',
  'PV6READ']}

In [27]:
float(df_sch['CNTSTUID'].nunique())

85537.0

In [38]:
utils.pipeline(llm, 'How many countries where in the survey?', df_sch, meta_sch, cols_retriever)

- Country ID
- Country Name
hint - Country ID
['CNTSTUID', 'CNTSCHID', 'CNT', 'SUBNATIO']
hint - Country Name
['CNT', 'REGION', 'COBN_F', 'OECD']
content="def run(df):\n    return float(df['CNT'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 285, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-62841e7b-131a-4de5-960a-3f53c96f0fdb-0' usage_metadata={'input_tokens': 285, 'output_tokens': 17, 'total_tokens': 302}
def run(df):
    return float(df['CNT'].dropna().nunique())


{'question': 'How many countries where in the survey?',
 'result': 80.0,
 'hint_cols': ['CNTSTUID',
  'CNTSCHID',
  'CNT',
  'SUBNATIO',
  'CNT',
  'REGION',
  'COBN_F',
  'OECD']}

In [12]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give linear regression coefficient as answer.', df_sch, meta_sch, cols_retriever)

1. Student ID
2. Social background (e.g. parental education level, household income)
3. Math score
hint 1. Student ID
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
hint 2. Social background (e.g. parental education level, household income)
['MISCED', 'FISCED', 'HISCED', 'ATTIMMP']
hint 3. Math score
['PV3MPRE', 'MATHPREF', 'PV3MPFS', 'PV3MPIN']
content="def run(data):\n    data = data[['MISCED', 'FISCED', 'HISCED', 'PV3MPRE', 'MATHPREF', 'PV3MPFS', 'PV3MPIN']].dropna()\n    \n    X = data[['MISCED', 'FISCED', 'HISCED', 'MATHPREF']]\n    y = data['PV3MPRE']\n    \n    from sklearn.linear_model import LinearRegression\n    model = LinearRegression()\n    model.fit(X, y)\n    \n    return float(model.coef_[4])" response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 461, 'total_tokens': 580}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-59ceebc5-d35e-411b-928e-1897177e2339-0' usage_metadata={'input_tokens':

IndexError: index 4 is out of bounds for axis 0 with size 4

In [114]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give linear regression coefficient as answer.', df_sch, meta_sch, cols_retriever)

1. Student ID
2. Social background (e.g. parental education level, household income)
3. Math score

Linear regression coefficient: The coefficient will indicate the strength and direction of the relationship between social background and math score.
hint 1. Student ID
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
hint 2. Social background (e.g. parental education level, household income)
['MISCED', 'FISCED', 'HISCED', 'ATTIMMP']
['MISCED', 'FISCED', 'HISCED', 'ATTIMMP']
hint 3. Math score
['PV3MPRE', 'MATHPREF', 'PV3MPFS', 'PV3MPIN']
['PV3MPRE', 'MATHPREF', 'PV3MPFS', 'PV3MPIN']
hint 
['PV2SCIE', 'PV9SCIE', 'SISCO', 'PV2READ']
['PV2SCIE', 'PV9SCIE', 'SISCO', 'PV2READ']
hint Linear regression coefficient: The coefficient will indicate the strength and direction of the relationship between social background and math score.
['MATHMOT', 'PV1MCCR', 'PV4MCCR', 'PV2MCCR']
['MATHMOT', 'PV1MCCR', 'PV4MCCR', 'PV2MCCR']
['Intl. Student ID', 'Intl. School ID',

{'question': 'What is the relation between social background and math score of students? Please give linear regression coefficient as answer.',
 'result': 8.370679333883233,
 'hint_cols': ['CNTSTUID',
  'CNTSCHID',
  'AGE',
  'COBN_S',
  'MISCED',
  'FISCED',
  'HISCED',
  'ATTIMMP',
  'PV3MPRE',
  'MATHPREF',
  'PV3MPFS',
  'PV3MPIN',
  'PV2SCIE',
  'PV9SCIE',
  'SISCO',
  'PV2READ',
  'MATHMOT',
  'PV1MCCR',
  'PV4MCCR',
  'PV2MCCR']}

In [49]:
utils.pipeline(llm, 'How many OECD member countries where in the survey? ', df_sch, meta_sch, cols_retriever)

Country Name, OECD Member Status
hint Country Name
['CNT', 'REGION', 'COBN_F', 'OECD']
hint  OECD Member Status
['OECD', 'ESCS', 'BMMJ1', 'CNTSTUID']
content="def run(df):\n    return float(df['OECD'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 281, 'total_tokens': 299}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a50936ba-b226-44a3-b52e-fc50ac667fd7-0' usage_metadata={'input_tokens': 281, 'output_tokens': 18, 'total_tokens': 299}
def run(df):
    return float(df['OECD'].dropna().nunique())


{'question': 'How many OECD member countries where in the survey? ',
 'result': 2.0,
 'hint_cols': ['CNT',
  'REGION',
  'COBN_F',
  'OECD',
  'OECD',
  'ESCS',
  'BMMJ1',
  'CNTSTUID']}

In [48]:
utils.pipeline(llm, 'How many countries that are OECD members where in the survey? ', df_sch, meta_sch, cols_retriever)

Country Name, OECD Member
hint Country Name
['CNT', 'REGION', 'COBN_F', 'OECD']
hint  OECD Member
['OECD', 'REGION', 'CNTSTUID', 'CNTSCHID']
content="def run(df):\n    return float(df['OECD'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 262, 'total_tokens': 280}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-e0f70ddc-3818-4a75-9382-dd3c7c6e6458-0' usage_metadata={'input_tokens': 262, 'output_tokens': 18, 'total_tokens': 280}
def run(df):
    return float(df['OECD'].dropna().nunique())


{'question': 'How many countries that are OECD members where in the survey? ',
 'result': 2.0,
 'hint_cols': ['CNT',
  'REGION',
  'COBN_F',
  'OECD',
  'OECD',
  'REGION',
  'CNTSTUID',
  'CNTSCHID']}

In [50]:
utils.pipeline(llm, 'how many countries that are OECD memeber were involved in the survey?', df_sch, meta_sch, cols_retriever)

Country Name, OECD Member
hint Country Name
['CNT', 'REGION', 'COBN_F', 'OECD']
hint  OECD Member
['OECD', 'REGION', 'CNTSTUID', 'CNTSCHID']
content="def run(df):\n    return df['CNT'][df['OECD'].dropna().astype(int) == 1].nunique() * 1.0" response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 263, 'total_tokens': 296}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f0bb32c7-fd08-4804-a311-aa3e3494fd98-0' usage_metadata={'input_tokens': 263, 'output_tokens': 33, 'total_tokens': 296}
def run(df):
    return df['CNT'][df['OECD'].dropna().astype(int) == 1].nunique() * 1.0


{'question': 'how many countries that are OECD memeber were involved in the survey?',
 'result': 37.0,
 'hint_cols': ['CNT',
  'REGION',
  'COBN_F',
  'OECD',
  'OECD',
  'REGION',
  'CNTSTUID',
  'CNTSCHID']}

In [51]:
utils.pipeline(llm, 'how many countries (column name CNT) that are memeber of OECD organisation were involved in the survey?', df_sch, meta_sch, cols_retriever)

Country Name, OECD Member Status, Survey Participation
hint Country Name
['CNT', 'REGION', 'COBN_F', 'OECD']
hint  OECD Member Status
['OECD', 'ESCS', 'BMMJ1', 'CNTSTUID']
hint  Survey Participation
['Option_TQ', 'Option_PQ', 'Option_WBQ', 'Option_ICTQ']
content="def run(df):\n    return float(df['CNT'][df['OECD'].dropna() == 1].nunique())" response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 394, 'total_tokens': 420}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b8091720-3e01-4bd3-896c-5cdb0a1c8ffc-0' usage_metadata={'input_tokens': 394, 'output_tokens': 26, 'total_tokens': 420}
def run(df):
    return float(df['CNT'][df['OECD'].dropna() == 1].nunique())


{'question': 'how many countries (column name CNT) that are memeber of OECD organisation were involved in the survey?',
 'result': 37.0,
 'hint_cols': ['CNT',
  'REGION',
  'COBN_F',
  'OECD',
  'OECD',
  'ESCS',
  'BMMJ1',
  'CNTSTUID',
  'Option_TQ',
  'Option_PQ',
  'Option_WBQ',
  'Option_ICTQ']}

In [52]:
utils.pipeline(llm, 'how many different countries that are memeber of OECD organisation were involved in the survey?', df_sch, meta_sch, cols_retriever)

Country, OECD_member
hint Country
['REGION', 'OECD', 'COBN_F', 'CNT']
hint  OECD_member
['OECD', 'REGION', 'OCOD1', 'BMMJ1']
content="def run(data):\n    return len(data['OECD'].dropna().unique())" response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 281, 'total_tokens': 298}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-0974d546-b1ff-4c03-98b3-de512a7d1a92-0' usage_metadata={'input_tokens': 281, 'output_tokens': 17, 'total_tokens': 298}
def run(data):
    return len(data['OECD'].dropna().unique())


{'question': 'how many different countries that are memeber of OECD organisation were involved in the survey?',
 'result': 2,
 'hint_cols': ['REGION',
  'OECD',
  'COBN_F',
  'CNT',
  'OECD',
  'REGION',
  'OCOD1',
  'BMMJ1']}

In [53]:
utils.pipeline(llm, 'In how many languages ​​was the questionnaire filled out?', df_sch, meta_sch, cols_retriever)

Language, Questionnaire ID
hint Language
['LANGTEST_COG', 'LANGN', 'LANGTEST_QQQ', 'LANGTEST_PAQ']
hint  Questionnaire ID
['BOOKID', 'LANGTEST_QQQ', 'Option_TQ', 'Option_PQ']
content="def run(data):\n    return float(data['LANGTEST_QQQ'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 282, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9dbe103c-bc1e-4eb8-8e10-4eff083d4210-0' usage_metadata={'input_tokens': 282, 'output_tokens': 20, 'total_tokens': 302}
def run(data):
    return float(data['LANGTEST_QQQ'].dropna().nunique())


{'question': 'In how many languages \u200b\u200bwas the questionnaire filled out?',
 'result': 55.0,
 'hint_cols': ['LANGTEST_COG',
  'LANGN',
  'LANGTEST_QQQ',
  'LANGTEST_PAQ',
  'BOOKID',
  'LANGTEST_QQQ',
  'Option_TQ',
  'Option_PQ']}

In [54]:
utils.pipeline(llm, 'In which country name have students the most books at home?', df_sch, meta_sch, cols_retriever)

Country name, Number of books at home
hint Country name
['CNT', 'REGION', 'COBN_F', 'OECD']
hint  Number of books at home
['PA195Q01JA', 'ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA']
content="def run(data):\n    data.dropna(subset=['PA195Q01JA', 'ST255Q01JA'], inplace=True)\n    data['total_books'] = data['PA195Q01JA'] + data['ST255Q01JA']\n    max_books_country = data.loc[data['total_books'].idxmax()]['CNT']\n    return max_books_country" response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 314, 'total_tokens': 388}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4422aaf1-15cb-43f7-b330-e06df1e73117-0' usage_metadata={'input_tokens': 314, 'output_tokens': 74, 'total_tokens': 388}
def run(data):
    data.dropna(subset=['PA195Q01JA', 'ST255Q01JA'], inplace=True)
    data['total_books'] = data['PA195Q01JA'] + data['ST255Q01JA']
    max_books_country = data.loc[data['total_books'].idxmax()]['CNT']
    retur

{'question': 'In which country name have students the most books at home?',
 'result': 'BEL',
 'hint_cols': ['CNT',
  'REGION',
  'COBN_F',
  'OECD',
  'PA195Q01JA',
  'ST255Q01JA',
  'ST256Q10JA',
  'ST256Q06JA']}

In [45]:
cols_retriever.invoke('- Country')

[Document(page_content='REGION', metadata={'year': 2022, 'original_col_name': 'REGION'}),
 Document(page_content='OECD country', metadata={'year': 2022, 'original_col_name': 'OECD'}),
 Document(page_content="Student father's country of birth", metadata={'year': 2022, 'original_col_name': 'COBN_F'}),
 Document(page_content="Student mother's country of birth", metadata={'year': 2022, 'original_col_name': 'COBN_M'})]

In [63]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give linear regression coefficients as answer. Mathematics score is stored as plausible values. To calculate math score take the average of plausible values of mathematics.', df_sch, meta_sch, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['MATHMOT', 'MATHPREF', 'ST296Q01JA', 'ST296Q02JA']
content="```python\nimport pandas as pd\nfrom sklearn.linear_model import LinearRegression\n\ndef run(data):\n    data = data[['ESCS', 'MATHMOT']]\n    data = data.dropna()\n    \n    X = data[['ESCS']]\n    y = data['MATHMOT']\n    \n    model = LinearRegression()\n    model.fit(X, y)\n    \n    return float(model.coef_[0])\n```" response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 353, 'total_tokens': 438}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-30a3fca1-755e-4388-860f-873872491a3b-0' usage_metadata={'input_tokens': 353, 'output_tokens': 85, 'total_tokens': 438}

import pandas as pd
from sklearn.linear_model import LinearRegression

def run(data):
    data = data[['ESCS', 'MATHMOT']]
    data = data.dropna()
    
    X = data[['ESC

{'question': 'What is the relation between social background and math score of students? Please give linear regression coefficients as answer. Mathematics score is stored as plausible values. To calculate math score take the average of plausible values of mathematics.',
 'result': -0.0022778274099931424,
 'hint_cols': ['ESCS',
  'SOCONPA',
  'COBN_M',
  'COBN_F',
  'MATHMOT',
  'MATHPREF',
  'ST296Q01JA',
  'ST296Q02JA']}

In [65]:
llm.invoke('I am analysing OECD PISA assessment results. In this dataset math scores are stored as "plausible values". It means, that each student has 10 different value, and their average should be used. social background has a specific index.  I want to find the answer for the question "What is the relation between social background and math score of students?" Can you generate a prompt that makes an LLM to answer the question? ')

AIMessage(content='"Please analyze the OECD PISA assessment results dataset to determine the relationship between social background index and math scores of students. Utilize the plausible values for math scores and consider the average of the 10 values for each student. Provide insights on how social background influences math performance among students."', response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 96, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0cefd587-0aa0-4cf4-a617-c43ba9f8ec48-0', usage_metadata={'input_tokens': 96, 'output_tokens': 57, 'total_tokens': 153})

In [67]:
utils.pipeline(llm, 'Please analyze the OECD PISA assessment results dataset to determine the relationship between social background index and math scores of students. Utilize the plausible values for math scores and consider the average of the 10 values for each student. Provide insights on how social background influences math performance among students.', df_sch, meta_sch, cols_retriever)

social background index, math scores, student ID
hint social background index
['ESCS', 'IMMIG', 'STUBMI', 'BMMJ1']
hint  math scores
['MATHMOT', 'MATHPREF', 'IC176Q01JA', 'ST331Q02JA']
hint  student ID
['CNTSTUID', 'CNTSCHID', 'AGE', 'BOOKID']
content="def run(data):\n    data = data[['ESCS', 'MATHMOT']]\n    data = data.dropna()\n    \n    avg_math_scores = data['MATHMOT'].mean()\n    avg_escs = data['ESCS'].mean()\n    \n    return avg_math_scores / avg_escs" response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 493, 'total_tokens': 555}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6e7a8180-6c02-44bd-b3d9-c47c582067d5-0' usage_metadata={'input_tokens': 493, 'output_tokens': 62, 'total_tokens': 555}
def run(data):
    data = data[['ESCS', 'MATHMOT']]
    data = data.dropna()
    
    avg_math_scores = data['MATHMOT'].mean()
    avg_escs = data['ESCS'].mean()
    
    return avg_math_scores / avg_e

{'question': 'Please analyze the OECD PISA assessment results dataset to determine the relationship between social background index and math scores of students. Utilize the plausible values for math scores and consider the average of the 10 values for each student. Provide insights on how social background influences math performance among students.',
 'result': -0.17518989987969524,
 'hint_cols': ['ESCS',
  'IMMIG',
  'STUBMI',
  'BMMJ1',
  'MATHMOT',
  'MATHPREF',
  'IC176Q01JA',
  'ST331Q02JA',
  'CNTSTUID',
  'CNTSCHID',
  'AGE',
  'BOOKID']}

In [ ]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give linear regression coefficients as answer. Mathematics score is stored as plausible values. To calculate math score take the average of plausible values of mathematics.', df_sch, meta_sch, cols_retriever)